In [ ]:
import os
import socket

In [ ]:
hostname = socket.gethostname()
if hostname == "lcls-srv04":
    os.environ["OMP_NUM_THREADS"] = str(1)
elif hostname == "test-rhel7":
    os.environ["OMP_NUM_THREADS"] = str(6)

In [ ]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("../../")
    sys.path.append("../")

run_dir = "/home/physics3/ml_tuning/20231120_LCLS_Injector/"
if not os.path.exists(run_dir):
    os.makedirs(run_dir)
print(sys.path)

## Set up image diagnostic

In [ ]:
import matplotlib.pyplot as plt
import yaml
from scripts.image import ImageDiagnostic

In [ ]:
fname = "../OTR3_config.yml"
image_diagnostic = ImageDiagnostic.parse_obj(yaml.safe_load(open(fname)))
image_diagnostic.save_image_location = run_dir
image_diagnostic.n_fitting_restarts = 2
image_diagnostic.visualize = False
image_diagnostic.background_file = run_dir + "OTRS_IN20_621_background.npy"
print(image_diagnostic.yaml())

In [ ]:
#image_diagnostic.measure_background(file_location=run_dir)

In [ ]:
image_diagnostic.background_file

In [ ]:
plt.imshow(image_diagnostic.background_image)

In [ ]:
image_diagnostic.test_measurement()

## Define VOCS

In [ ]:
import pandas as pd
from xopt import VOCS

In [ ]:
variables = ["SOLN:IN20:121:BCTRL", "QUAD:IN20:121:BCTRL"]
# variables = [
#     "SOLN:IN20:121:BCTRL", "QUAD:IN20:121:BCTRL", "QUAD:IN20:122:BCTRL",
#     "QUAD:IN20:361:BCTRL", "QUAD:IN20:371:BCTRL", "QUAD:IN20:425:BCTRL",
#     "QUAD:IN20:441:BCTRL", "QUAD:IN20:511:BCTRL", "QUAD:IN20:525:BCTRL",
# ]

img_constraints = {
    "bb_penalty": ["LESS_THAN", 0.0],
    "log10_total_intensity": ["GREATER_THAN", image_diagnostic.min_log_intensity]
}

filename = "../variables.csv"
variable_ranges = pd.read_csv(filename, index_col=0, header=None).T.to_dict(orient='list')
vocs = VOCS(
    variables = {ele: variable_ranges[ele] for ele in variables},
    objectives = {"total_size": "MINIMIZE"},
    constraints = img_constraints,
)
print(vocs.as_yaml())

## Define NN prior

In [ ]:
sys.path.append("calibration/calibration_modules/")
from decoupled_linear import OutputOffset, DecoupledLinearOutput
from utils import load_model

In [ ]:
objective_model = load_model(
    input_variables=vocs.variable_names,
    model_path="lcls_cu_injector_nn_model/",
)
lume_model = objective_model.model.model

# define miscalibrated objective model
y_size = len(vocs.objective_names)
miscal_model = DecoupledLinearOutput(
    model=objective_model,
    y_offset_initial=torch.full((y_size,), -0.5),
    y_scale_initial=torch.ones(y_size),
)
miscal_model.requires_grad_(False);

# define prior mean
prior_mean = OutputOffset(
    model=miscal_model,
)

## Define evaluate function

In [ ]:
import numpy as np
import torch
from time import sleep
from epics import caput, caget_many, caget
from utils import get_model_predictions, numpy_save

In [ ]:
def evaluate(input_dict, generator = None):
    global image_diagnostic
    # set PVs
    for k, v in input_dict.items():
        print(f'CAPUT {k} {v}')
        caput(k, v)

    sleep(5.0)

    # get beam sizes from image diagnostic
    metadata = input_dict
    results = image_diagnostic.measure_beamsize(1, **metadata)
    results["S_x_mm"] = np.array(results["Sx"]) * 1e-3
    results["S_y_mm"] = np.array(results["Sy"]) * 1e-3

    # get other PV's NOTE: Measurements not synchronous with beamsize measurements!
    results = results

    # add total beam size
    sigma_xy = np.sqrt(np.array(results["Sx"]) ** 2 + np.array(results["Sy"]) ** 2)
    roundness = np.abs(np.array(results["Sx"]) - np.array(results["Sy"]))
    results["sigma_xy"] = sigma_xy
    results["total_size"] = objective_model.objective_scale * (sigma_xy + roundness)
    # results["total_size"] = np.sqrt(np.abs(np.array(results["Sx"])) * np.array(results["Sy"]))
    
    # GP model predictions
    model_predictions = get_model_predictions(input_dict, generator)
    results.update(model_predictions)

    numpy_save()
    
    return results

In [ ]:
# define custom mean
objective_model.requires_grad_(False);

custom_mean = objective_model
# custom_mean = OutputOffset(
#     model=objective_model,
# )

## Adjust variable ranges

In [ ]:
vocs.variables = {
    k: lume_model.input_variables[lume_model.input_names.index(k)].value_range 
    for k in vocs.variable_names
}
vocs.variables["SOLN:IN20:121:BCTRL"] = [0.467, 0.479]
print(vocs.as_yaml())

In [ ]:
# from xopt.utils import get_local_region

# # get current point
# current_value = {
#     'SOLN:IN20:121:BCTRL': 0.4809822,
#     'QUAD:IN20:121:BCTRL': 0.0018092622,
#     'QUAD:IN20:122:BCTRL': -0.0110517,
#     'QUAD:IN20:361:BCTRL': -3.37538,
#     'QUAD:IN20:371:BCTRL': 2.55894,
#     'QUAD:IN20:425:BCTRL': -1.11579,
#     'QUAD:IN20:441:BCTRL': -0.11462,
#     'QUAD:IN20:511:BCTRL': 3.4887333,
#     'QUAD:IN20:525:BCTRL': -2.887897,
# }

# # get small region around current point to sample
# random_sample_region = get_local_region(current_value, vocs, fraction=0.4)
# random_sample_region

In [ ]:
# clamped_vars = {}
# for k, v in random_sample_region.items():
#     clamped_vars[k] = [
#         np.max([random_sample_region[k][0], vocs.variables[k][0]]),
#         np.min([random_sample_region[k][1], vocs.variables[k][1]]),
#     ]
# vocs.variables = clamped_vars
# print(vocs.as_yaml())

## Run Xopt

In [ ]:
from xopt import Xopt, VOCS
from xopt.evaluator import Evaluator
from xopt.numerical_optimizer import LBFGSOptimizer
from xopt.generators.bayesian import ExpectedImprovementGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor

In [ ]:
# remember to set use low noise prior to false!!!
gp_constructor = StandardModelConstructor(
    use_low_noise_prior=False,
    mean_modules={vocs.objective_names[0]: prior_mean},
    trainable_mean_keys=[vocs.objective_names[0]],
)
generator = ExpectedImprovementGenerator(
    vocs=vocs,
    gp_constructor=gp_constructor,
)
generator.numerical_optimizer.max_iter = 200
evaluator = Evaluator(function=evaluate, function_kwargs={"generator": None})
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)

# pass generator to evaluator to compute model predictions
X.evaluator = Evaluator(function=evaluate, function_kwargs={"generator": X.generator})

# define dump file
dump_file = run_dir + "optimize_1.yml"
if os.path.isfile(dump_file):
    print("Dump file exists already!")
X.dump_file = dump_file
X

In [ ]:
n_init = 10
initial_data_file = os.path.join(run_dir, f"optimize_initial_data_n={n_init}.csv")

if os.path.isfile(initial_data_file):
    initial_data = pd.read_csv(initial_data_file)
    X.add_data(initial_data)
else:
    X.random_evaluate(n_init)
    X.data.to_csv(initial_data_file, index=False)

X.data

In [ ]:
%%time
for i in range(10):
    print(i)
    X.step()

## Display results

In [ ]:
from utils import get_running_optimum

In [ ]:
opt = get_running_optimum(
    data=X.data,
    objective_name=X.vocs.objective_names[0],
    maximize=X.vocs.objectives[X.vocs.objective_names[0]].upper() == "MAXIMIZE",
)
ax = X.data.plot(y=X.vocs.objective_names[0])
ax.plot(opt, label="running_optimum")
ax.legend();

In [ ]:
vocs.normalize_inputs(X.data).plot(y=X.vocs.variable_names);

In [ ]:
X.generator.computation_time[["training", "acquisition_optimization"]].plot();

In [ ]:
label_suffixes = ["", "_prior_mean", "_posterior_mean", "_posterior_sd"]
X.data[[X.vocs.objective_names[0] + k for k in label_suffixes]].plot();

In [ ]:
variable_names = X.vocs.variable_names
if X.vocs.n_variables not in [1, 2]:
    variable_names = ["SOLN:IN20:121:BCTRL"]

X.generator.visualize_model(
    variable_names=variable_names,
    show_prior_mean=True,
);